In [1]:
import seaborn as sns
import metapack as mp
import pandas as pd
import numpy as np
import geopandas as gpd

import matplotlib.pyplot as plt
from IPython.display import display 

import datetime
from cityiq import Config, CityIq
from cityiq.api import Asset, Location

%matplotlib inline
sns.set_context('notebook')
mp.jupyter.init()


In [2]:
#pkg = mp.jupyter.open_package()
pkg = mp.jupyter.open_source_package()
pkg

# San Diego City IQ Assets and Locations
`cityiq.io-objects-san_diego-5` Last Update: 2019-06-04T20:19:25

_All assets types extracted form the San Diego City Iq system_


These datafile are extracts of the assets and locations from the San Diego
CityIQ system. Refer to the [CityIQ developer
documentation](https://ie-cities-docs.run.aws-usw02-pr.ice.predix.io/#c_overview
 _of_general_apis.html) for details about these data records. The data are
extracted using the [cityiq Python package](https://pypi.org/project/cityiq/).
See the ExtractAssets.ipynb notebook for the extract process.

## Versions

1. Development version
2. Development version
3. First release
4. Added tract and community names
5. Rebuild with recent data, and broke out asset and location types


 
 
## Contacts

* **Wrangler** [Eric Busboom](mailto:eric@civicknowledge.com), [Civic Knowledge](http://civicknowledge.com)
 
## Resources

* ** [assets](notebooks/ExtractAssets.ipynb#assets)**. All assets
* ** [locations](notebooks/ExtractAssets.ipynb#locations)**. All locations: walkways, traffic lanes and parking zones
* ** [walkways](notebooks/ExtractAssets.ipynb#walkways)**. Walkways
* ** [traffic_lanes](notebooks/ExtractAssets.ipynb#traffic_lanes)**. Traffic lanes
* ** [parking_zones](notebooks/ExtractAssets.ipynb#parking_zones)**. Parking zones
* ** [cameras](notebooks/ExtractAssets.ipynb#cameras)**. Cameras
* ** [mics](notebooks/ExtractAssets.ipynb#mics)**. Microphones
* ** [nodes](notebooks/ExtractAssets.ipynb#nodes)**. Nodes
* ** [em_sensors](notebooks/ExtractAssets.ipynb#em_sensors)**. Electro-magnetic sensors
* ** [env_sensors](notebooks/ExtractAssets.ipynb#env_sensors)**. Environmental sensors
 
## References

* **[sdroads](metapack+http://library.metatab.org/sangis.org-roads-sandiego-1.zip#all_roads)**. Roads in San Diego
* **[tract_boundaries](metapack+http://library.metatab.org/sandiegodata.org-geography-2018.csv#tract_boundaries)**. San Diego tracts and communities
* **[sd_community_boundaries](metapack+http://library.metatab.org/sandiegodata.org-geography-2018.csv#sd_community_boundaries)**. San Diego tracts and communities

In [3]:
tracts = pkg.reference('tract_boundaries').geoframe()
comm = pkg.reference('sd_community_boundaries').geoframe()


In [4]:

tz = datetime.datetime.now(datetime.timezone.utc).astimezone().tzinfo

config = Config()
ciq = CityIq(config, start_time='20180901')

In [ ]:
roads = pkg.reference('sdroads').geoframe()
sdroads = roads[(roads.lpsjur == 'SD') | (roads.rpsjur == 'SD')].copy()

# Convert to EPSG:2875 ( California zone 6, feet ) so we can make the buffer in feet
sdroads = sdroads.to_crs({'init':'epsg:2875'})

# Buffer the roads to be big enough to enclose the assets
sdroads['geometry'] = sdroads.geometry.apply(lambda g : g.buffer(60,cap_style=1, join_style=2))

sdroads = sdroads.to_crs( {'init':'epsg:4326'} )
# sdroads.to_csv('sdroads.csv')

In [ ]:
assets = gpd.GeoDataFrame(ciq.asset_dataframe, geometry='geometry')
l1 = len(assets)
print(len(assets))
assets.crs = {'init':'epsg:4326'} 

ac = assets.copy()
ac['geometry'] = ac.centroid.to_crs({'init':'epsg:2875'}).buffer(20).to_crs( {'init':'epsg:4326'} )

# Link to roads
t = gpd.sjoin(ac, sdroads, how='left').drop(columns=['index_right'])

# Link to communities
t = gpd.sjoin(t, comm[comm['type'] == 'sd_community'][['geometry','name']], how='left').drop(columns=['index_right'])

# Link to tracts
t = gpd.sjoin(t, tracts[['geometry','geoid']], how='left')

t = t[list(assets)[:-1] + 
           ['name','geoid','roadsegid', 'speed', 'oneway','abloaddr', 'abhiaddr', 'rd30full'] +
           ['geometry'] ].drop_duplicates(subset='assetUid')\
           .rename(columns={'name':'community_name','geoid':'tract_geoid'})

assets = t.drop(columns='geometry').join(assets['geometry'])

assert len(assets) == l1
           
# How many assets and locations did not get linked to roads?
len(assets[assets.speed.isnull()]), len(assets)
           

In [ ]:
locations = gpd.GeoDataFrame(ciq.locations_dataframe, geometry='geometry')

l1 = len(locations)
locations.crs = {'init':'epsg:4326'}
lc = locations.copy()

lc['geometry'] =  locations.centroid.to_crs({'init':'epsg:2875'}).buffer(20).to_crs( {'init':'epsg:4326'} )

# Link to roads
t = gpd.sjoin(lc, sdroads, how='left').drop(columns=['index_right'])

# Link to communities
t = gpd.sjoin(t, comm[comm['type'] == 'sd_community'][['geometry','name']], how='left').drop(columns=['index_right'])

# Link to tracts
t = gpd.sjoin(t, tracts[['geometry','geoid']], how='left')

t = t[list(lc)[:-1] + 
           ['name','geoid','roadsegid', 'speed', 'oneway','abloaddr', 'abhiaddr', 'rd30full'] +
           ['geometry'] ].drop_duplicates(subset='locationUid')\
           .rename(columns={'name':'community_name','geoid':'tract_geoid'})

locations = t.drop(columns='geometry').join(locations['geometry'])


assert len(locations) == l1

In [7]:
locations.head()

,locationUid,locationType,parentLocationUid,geometry
0,004361eb,WALKWAY,004361eb,LINESTRING (-117.1706148251287 32.726548905684...
1,00456472,TRAFFIC_LANE,00456472,LINESTRING (-117.1705245131968 32.726558517207...
2,0051796c,WALKWAY,0051796c,LINESTRING (-117.1666581391174 32.722214661862...
3,00537bf3,TRAFFIC_LANE,00537bf3,LINESTRING (-117.166581746989 32.7222178855665...
4,005f90ed,WALKWAY,005f90ed,LINESTRING (-117.1665182431568 32.723810889902...


In [8]:
locations.locationType.unique()

array(['WALKWAY', 'TRAFFIC_LANE', 'PARKING_ZONE'], dtype=object)

In [9]:
walkways = locations[locations.locationType == 'WALKWAY']
traffic_lanes = locations[locations.locationType == 'TRAFFIC_LANE']
parking_zones = locations[locations.locationType == 'PARKING_ZONE']


In [10]:
assets.assetType.unique()

array(['CAMERA', 'MIC', 'NODE', 'EM_SENSOR', 'ENV_SENSOR'], dtype=object)

In [11]:
nodes = assets[assets.assetType == 'NODE']
cameras = assets[assets.assetType == 'CAMERA']
mics = assets[assets.assetType == 'MIC']
em_sensors = assets[assets.assetType == 'EM_SENSOR']
env_sensors = assets[assets.assetType == 'ENV_SENSOR']


In [12]:
for e in list(locations.locationType.unique())+list(assets.assetType.unique()):
    print (e.lower())

walkway
traffic_lane
parking_zone
camera
mic
node
em_sensor
env_sensor
